In [1]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np

import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

from tensorflow import keras
model = keras.models.load_model('lstm_model_5.keras')

# 데이터프레임 읽기
df = pd.read_csv('./weather_kor4.csv')

input_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']
output_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']

# 데이터 전처리 변수마다 범위가 다르므로 다르게 적용
from sklearn.preprocessing import MinMaxScaler
scalers = {}
for feature in input_features:
    scaler = MinMaxScaler()
    df[feature] = scaler.fit_transform(df[[feature]])
    # print(scaled_data)
    scalers[feature] = scaler
    
# last_sequence =df[input_features].iloc[-720:, :] 
# next_day_predictions = []
# for i in range(72):
#     next_day_prediction = model.predict(np.array([last_sequence]))
#     for feature, scaler in scalers.items():
#         if feature in output_features:
#             next_day_prediction[0, output_features.index(feature)] = scaler.inverse_transform(next_day_prediction[0, output_features.index(feature)].reshape(-1, 1)).flatten()
#     next_day_predictions.append(next_day_prediction.flatten())
#     last_sequence = np.vstack([last_sequence[1:], next_day_prediction])
# print("72시간 동안의 기상 예측:")
# for i, prediction in enumerate(next_day_predictions, start=1):
#     print(f"시간 {i}: {prediction}")


2024-05-28 13:50:52.875825: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 13:50:53.059051: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 13:50:53.691313: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 13:50:55.243855: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
import gradio as gr
import numpy as np
import pandas as pd
import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler


model = tf.keras.models.load_model('./lstm_model_5.keras')
scaler_data = pd.read_csv('./weather_kor4.csv')


input_features = ['기온', '강수량', '풍속_num', '풍향_num', '습도', '현지기압', '전운량', 'W1', 'W2']
output_features = input_features.copy()
FUTURE_TARGET = 48



scalers = {}
for feature in input_features:
    scaler = MinMaxScaler()
    scaler.fit(scaler_data[[feature]])
    scalers[feature] = scaler


dropdown_options = []

# 마지막 데이터의 날짜를 입력 이 이후로 1시간씩 예측하므로
last_date = pd.to_datetime(scaler_data['날짜'].iloc[-1])
for i in range(FUTURE_TARGET):
    # 한시간씩 datetime으로 저장
    future_date_str = (last_date + pd.Timedelta(hours=i)).strftime('%Y-%m-%d %H:%M')
    dropdown_options.append(future_date_str)

def preprocess_data(date_time):
    
    future_date = pd.to_datetime(date_time)
    # 720시간의 앞부분을 예측하겠다 2023-05-23 12:00 씩을 예측할때 2023-04-23 12:00 ~ 2023-05-23 11:00 까지를 보겠다
    sequence_length = 720
    last_sequence = scaler_data[input_features].iloc[-sequence_length:, :].values

    
    next_day_predictions = []
    for _ in range(FUTURE_TARGET):
        # 720시간 데이터를 input으로 예측
        next_day_prediction = model.predict(np.array([last_sequence]))
        
        for i, feature in enumerate(output_features):
            # 나온 예측값들을 각각의 스케일러에 맞게 역변환해주기 1차원배열로 저장
            next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
        next_day_predictions.append(next_day_prediction.flatten())
        # 그리고 그 추가된 값을 last_sequence vstack 하여 쌓기 - 여전히 720개 유지
        last_sequence = np.vstack([last_sequence[1:], next_day_prediction])
        print(last_sequence)

    # 데이터프레임으로 만들고
    predictions_df = pd.DataFrame(next_day_predictions, columns=output_features)
    future_timestamps = pd.date_range(start=last_date + pd.Timedelta(hours=1), periods=FUTURE_TARGET, freq='H')
    predictions_df.index = future_timestamps

    
    selected_prediction = predictions_df.loc[future_date.strftime('%Y-%m-%d %H:%M:%S')]

    return selected_prediction.to_frame().T

iface = gr.Interface(
    fn=preprocess_data,
    inputs=gr.Dropdown(choices=dropdown_options, label="Select Date and Time"),
    outputs=gr.Dataframe(headers=output_features, type="pandas"),
    title="Future Weather Forecast",
    description="Select the date and time to see the weather forecast for that specific time.",
    allow_flagging=False
)


iface.launch(share=True)



2024-05-29 00:05:46.973649: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 00:05:47.132666: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 00:05:47.722152: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 00:05:49.152001: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/encore/miniconda3/lib/python3.10/site-packages/gradio/interface.py:382: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f18d110c66e1b45a14.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
[[13.8         0.          4.         ... 10.          2.
   1.        ]
 [13.5         0.          4.1        ... 10.          2.
   1.        ]
 [13.3         0.          4.1        ...  8.          0.
   1.        ]
 ...
 [17.7         0.          2.9        ...  0.          1.
   1.        ]
 [16.8         0.          3.1        ...  0.          1.
   1.        ]
 [24.43170929 -2.09895825  6.74515486 ...  1.22957146  0.35296282
   0.99944347]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
[[13.5         0.          4.1        ... 10.          2.
   1.        ]
 [13.3         0.          4.1        ...  8.          0.
   1.        ]
 [13.2         0.          3.1        ... 10.          2.
   1.        ]
 ...
 [16.8         0.          3.1        ...  0.          1.
   1.        ]
 [24.43170929 -2.09895825  6.74515486 ...  1.22957146  0.35296282
   0.99944347]
 [27.35920715 -1.62868392  7.86969519 ...  0.29484764  0.43329394
   1.0048672 ]]


/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
[[13.3         0.          4.1        ...  8.          0.
   1.        ]
 [13.2         0.          3.1        ... 10.          2.
   1.        ]
 [12.3         0.5         2.5        ... 10.          2.
   2.        ]
 ...
 [24.43170929 -2.09895825  6.74515486 ...  1.22957146  0.35296282
   0.99944347]
 [27.35920715 -1.62868392  7.86969519 ...  0.29484764  0.43329394
   1.0048672 ]
 [28.07216263 -1.61810684  8.34183502 ... -0.77348262  0.49756438
   1.02269495]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
[[13.2         0.          3.1        ... 10.          2.
   1.        ]
 [12.3         0.5         2.5        ... 10.          2.
   2.        ]
 [11.8         2.7         2.9        ... 10.          2.
   2.        ]
 ...
 [27.35920715 -1.62868392  7.86969519 ...  0.29484764  0.43329394
   1.0048672 ]
 [28.07216263 -1.61810684  8.34183502 ... -0.77348262  0.49756438
   1.02269495]
 [28.66048431 -1.62322259  8.74248123 ... -1.15467334  0.58229595
   

...

/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
[[14.6         0.          1.5        ...  0.          1.
   1.        ]
 [14.          0.          1.1        ...  0.          1.
   1.        ]
 [13.6         0.          0.4        ...  0.          1.
   1.        ]
 ...
 [24.32739258 -1.44362402  9.33810997 ... -1.6143018   0.71522248
   1.11624491]
 [24.32612801 -1.44358981  9.33803654 ... -1.61427617  0.71521229
   1.11624241]
 [24.32506371 -1.44356084  9.33797169 ... -1.61425352  0.71520334
   1.11624026]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
[[14.          0.          1.1        ...  0.          1.
   1.        ]
 [13.6         0.          0.4        ...  0.          1.
   1.        ]
 [13.3         0.          0.7        ...  0.          1.
   1.        ]
 ...
 [24.32612801 -1.44358981  9.33803654 ... -1.61427617  0.71521229
   1.11624241]
 [24.32506371 -1.44356084  9.33797169 ... -1.61425352  0.71520334
   1.11624026]
 [24.3241806  -1.44353616  9.33791924 ... -1.61423624  0.71519601
  

/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
[[13.6         0.          0.4        ...  0.          1.
   1.        ]
 [13.3         0.          0.7        ...  0.          1.
   1.        ]
 [12.8         0.          0.3        ...  0.          1.
   1.        ]
 ...
 [24.32506371 -1.44356084  9.33797169 ... -1.61425352  0.71520334
   1.11624026]
 [24.3241806  -1.44353616  9.33791924 ... -1.61423624  0.71519601
   1.11623847]
 [24.32343674 -1.44351602  9.3378706  ... -1.61421955  0.71518958
   1.11623693]]


/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_day_prediction[0, i] = scalers[feature].inverse_transform(next_day_prediction[:, i].reshape(-1, 1)).flatten()
/tmp/ipykernel_1093/888311811.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy

In [ ]:
last_sequence